# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
# import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
from data_processing import DataProcessing
from clean_predictions import PredictionDataCleaner

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/detraviousjamaribrinkley/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [4]:
load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.9796735644340515

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8603723049163818

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8691261410713196

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8487552404403687

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8647665977478027

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8541026711463928

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8756679892539978



/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_71118/312081125.py:18: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


In [5]:
# pom_mappings

In [6]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.979674
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.860372
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.869126
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.848755
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.864767
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.854103
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.875668


Comparison
1. No focus on #s, more on words

In [11]:
prediction_doc = load_nlp_model("11%")
observation_doc = load_nlp_model("11.5%")
observation_doc_2 = load_nlp_model("12%")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_71118/2383531039.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.9415925741195679

In [12]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_71118/4256136358.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc_2)


0.9456990361213684

In [ ]:
prediction_doc = load_nlp_model("Apple's stock rose by 10%.")
observation_doc = load_nlp_model("Apple's stock rose by 11%.")
similarity = prediction_doc.similarity(observation_doc)
similarity

In [ ]:
pdv = prediction_doc.vector # 10
pdv

In [ ]:
odv = observation_doc.vector # 10.5
odv

[Five most popular similarity measures implementation in python](https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/)

In [ ]:
from math import*

def square_rooted(x):

    return round(sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x,y):

    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

cosine_similarity(pdv, odv)

In [ ]:
prediction_doc = load_nlp_model("10")
observation_doc = load_nlp_model("10.5")
observation_doc_2 = load_nlp_model("11")

similarity = prediction_doc.similarity(observation_doc)
similarity

In [ ]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example sentences
sentence1 = "Apple's stock rose by 10%."
sentence2 = "Apple's stock rose by 10.5%."

# Tokenize the sentences
tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

# Add [CLS] and [SEP] tokens
tokens = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2 + ['[SEP]']
print("Token:", tokens)

# Convert tokens to input IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)

# Display the tokens and input IDs
print("Input IDs:", input_ids)

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Convert tokens to input IDs
input_ids1 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens1)).unsqueeze(0)  # Batch size 1
input_ids2 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens2)).unsqueeze(0)  # Batch size 1

# Obtain the BERT embeddings
with torch.no_grad():
    outputs1 = model(input_ids1)
    outputs2 = model(input_ids2)
    embeddings1 = outputs1.last_hidden_state[:, 0, :]  # [CLS] token
    embeddings2 = outputs2.last_hidden_state[:, 0, :]  # [CLS] token

# Calculate similarity
similarity_score = cosine_similarity(embeddings1, embeddings2)
print("Similarity Score:", similarity_score)

In [ ]:
mappings = {
    "Prediction": ["I am predicting it will be sunny on 4 April 2025.", "Apple's stock will be rising by 10%.", "Apple's stock will be rising by 10%."],
    "Observation": ["It was sunny on 4 April 2025", "Apple's stock is rising by 900%.",  "Apple's stock rose by 900%."]
}

df = pd.DataFrame(mappings)
df


In [ ]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

clean_data = PredictionDataCleaner(df)

In [ ]:
updated_predictions_df = clean_data.lemmatize(col_name='Prediction', lem_col_name='Predictions Lem')
updated_predictions_df

In [ ]:
updated_observations_df = clean_data.lemmatize(col_name='Observation', lem_col_name='Observations Lem')
updated_observations_df

In [ ]:
lem_df = updated_predictions_df.merge(updated_observations_df)
lem_df

- Issues above are: 

    1. "'s" isn't joined to Apple
    2. It doesn't consider "will be rising" as a single future tense phrase, thus breaks it up into words (with only one future tense word of "rising")

In [ ]:
import nltk
nltk.download('wordnet')
  
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

In [ ]:
lem.lemmatize('predicting')

In [ ]:
lem.lemmatize('rising')

In [ ]:
lem.lemmatize('rocks')

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("predicting, predicts, rising, rises, rocks, rose, will be rising, is rising by.")
doc = nlp("I am predicting it will be sunny on 4 April 2025.")

words_lemmas_list = [token.lemma_ for token in doc]
print(words_lemmas_list)
